In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


# 🎟️SCRAPING TICKET MASTER

In [7]:
# 📥 Imports y configuración inicial
import os
import time
import requests
import pandas as pd
from random import random
from datetime import datetime, timedelta

API_KEY = 'AsIPMT0pnI4AdbyDWkMWRHBbaG2i8vba'  # ← poner aquí API key

BASE_URL = 'https://app.ticketmaster.com/discovery/v2/events.json'

# 📍 Definición de ciudades y radios de búsqueda

CITY_GEO = {
    'Santa Cruz de Tenerife': {'lat': 28.4636, 'lon': -16.2518, 'radius_km': 40},
    'Barcelona':              {'lat': 41.3874, 'lon':   2.1686,  'radius_km': 45},
    'Madrid':                 {'lat': 40.4168, 'lon':  -3.7038,  'radius_km': 50},
    'Malaga':                 {'lat': 36.7213, 'lon':  -4.4213,  'radius_km': 40},
    'Las Palmas de Gran Canaria': {'lat': 28.1235, 'lon': -15.4363, 'radius_km': 40},
    'Sevilla':                {'lat': 37.3891, 'lon':  -5.9845,  'radius_km': 45},
    'Valencia':               {'lat': 39.4699, 'lon':  -0.3763,  'radius_km': 45},
    'Palma de Mallorca':      {'lat': 39.5696, 'lon':   2.6502,  'radius_km': 35},
}

# ⏱️ Rango temporal de búsqueda: desde hoy hasta un año

start_dt = datetime.utcnow().strftime('%Y-%m-%dT00:00:00Z')
end_dt = (datetime.utcnow() + timedelta(days=365)).strftime('%Y-%m-%dT00:00:00Z')

# 🌐 Sesión de requests para la API

session = requests.Session()
session.headers.update({'Accept': 'application/json'})

# 🔄 Función para pedir una página a la API con reintentos

def fetch_page(params, max_retries=5):
    delay = 1.0
    for _ in range(max_retries):
        try:
            r = session.get(BASE_URL, params=params, timeout=20)
            if r.status_code in (429, 500, 502, 503, 504):
                time.sleep(delay + random())
                delay = min(delay * 2, 16)
                continue
            r.raise_for_status()
            return r.json()
        except requests.RequestException:
            time.sleep(delay + random())
            delay = min(delay * 2, 16)
    return None

# 📑 Función para descargar todas las páginas de resultados

def fetch_all(params_base):
    results = []
    page = 0
    while True:
        params = {**params_base, 'page': page, 'size': 200, 'sort': 'date,asc'}
        data = fetch_page(params)
        if not data:
            break
        events = (data.get('_embedded') or {}).get('events', [])
        results.extend(events)
        p = data.get('page') or {}
        if (p.get('number', 0) >= (p.get('totalPages', 1) - 1)):
            break
        page += 1
    return results

# 🧹 Normalización de cada evento en un diccionario plano

def normalize(e, ciudad_buscada):
    venue = (e.get('_embedded', {}).get('venues') or [{}])[0]
    address = venue.get('address', {}) or {}
    city_info = venue.get('city', {}) or {}
    country = venue.get('country', {}) or {}
    price = (e.get('priceRanges') or [{}])[0]
    classification = (e.get('classifications') or [{}])[0]

    return {
        'ciudad_buscada': ciudad_buscada,
        'id_evento': e.get('id'),
        'nombre': e.get('name'),
        'fecha': (e.get('dates') or {}).get('start', {}).get('localDate'),
        'hora': (e.get('dates') or {}).get('start', {}).get('localTime'),
        'estado_evento': (e.get('dates') or {}).get('status', {}).get('code'),
        'fecha_venta': (e.get('sales') or {}).get('public', {}).get('startDateTime'),
        'url': e.get('url'),
        'tipo_evento': classification.get('segment', {}).get('name'),
        'genero': classification.get('genre', {}).get('name'),
        'subgenero': classification.get('subGenre', {}).get('name'),
        'precio_min': price.get('min'),
        'precio_max': price.get('max'),
        'promotor': (e.get('promoter') or {}).get('name'),
        'ciudad_real': city_info.get('name'),
        'pais': country.get('name'),
        'lugar': venue.get('name'),
        'direccion': address.get('line1'),
        'latitud': (venue.get('location') or {}).get('latitude'),
        'longitud': (venue.get('location') or {}).get('longitude'),
        'codigo_postal': venue.get('postalCode'),
    }

# 🚀 Bucle principal: recorrer ciudades, combinar búsquedas city+geo y acumular resultados

resultados = []
vistos = set()  
contador_ciudades = {}  # ← nuevo dict para contar

for ciudad, geo in CITY_GEO.items():
    print(f"\n🏙️  {ciudad}: extracción combinada (city + geo)")

    params_city = {
        'apikey': API_KEY,
        'locale': '*',
        'countryCode': 'ES',
        'city': ciudad,
        'startDateTime': start_dt,
        'endDateTime': end_dt,
    }
    eventos_city = fetch_all(params_city)

    params_geo = {
        'apikey': API_KEY,
        'locale': '*',
        'countryCode': 'ES',
        'latlong': f"{geo['lat']},{geo['lon']}",
        'radius': geo['radius_km'],
        'unit': 'km',
        'startDateTime': start_dt,
        'endDateTime': end_dt,
    }
    eventos_geo = fetch_all(params_geo)

    total_ciudad = 0
    for e in (eventos_city + eventos_geo):
        eid = e.get('id')
        if not eid or eid in vistos:
            continue
        vistos.add(eid)
        resultados.append(normalize(e, ciudad))
        total_ciudad += 1

    contador_ciudades[ciudad] = total_ciudad
    print(f"   → nuevos únicos añadidos: {total_ciudad}")

# 💾 Guardar resultados en CSV y mostrar resumen

df = pd.DataFrame(resultados)
df.to_csv('eventos_ticketmaster.csv', index=False, encoding='utf-8')
print("\n✅ Archivo guardado: 'eventos_ticketmaster.csv'")
print(f"Total eventos únicos: {len(df)}")

# 📊 Mostrar contador por ciudad

print("\n📊 Eventos por ciudad (únicos):")
for ciudad, cuenta in contador_ciudades.items():
    print(f" - {ciudad}: {cuenta}")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_23860\1740471630.py:28: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_dt = datetime.utcnow().strftime('%Y-%m-%dT00:00:00Z')
C:\Users\Usuario\AppData\Local\Temp\ipykernel_23860\1740471630.py:29: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_dt = (datetime.utcnow() + timedelta(days=365)).strftime('%Y-%m-%dT00:00:00Z')



🏙️  Santa Cruz de Tenerife: extracción combinada (city + geo)
   → nuevos únicos añadidos: 2

🏙️  Barcelona: extracción combinada (city + geo)
   → nuevos únicos añadidos: 150

🏙️  Madrid: extracción combinada (city + geo)
   → nuevos únicos añadidos: 318

🏙️  Malaga: extracción combinada (city + geo)
   → nuevos únicos añadidos: 9

🏙️  Las Palmas de Gran Canaria: extracción combinada (city + geo)
   → nuevos únicos añadidos: 1

🏙️  Sevilla: extracción combinada (city + geo)
   → nuevos únicos añadidos: 17

🏙️  Valencia: extracción combinada (city + geo)
   → nuevos únicos añadidos: 26

🏙️  Palma de Mallorca: extracción combinada (city + geo)
   → nuevos únicos añadidos: 4

✅ Archivo guardado: 'eventos_ticketmaster.csv'
Total eventos únicos: 527

📊 Eventos por ciudad (únicos):
 - Santa Cruz de Tenerife: 2
 - Barcelona: 150
 - Madrid: 318
 - Malaga: 9
 - Las Palmas de Gran Canaria: 1
 - Sevilla: 17
 - Valencia: 26
 - Palma de Mallorca: 4
